<a href="https://colab.research.google.com/github/aabdulhakeem/Forward-and-backward-chaining/blob/main/forwardandbackward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
facts = {}
rules = {}

In [13]:
def convert_fact(line):
    fact = line.strip().split()
    if len(fact) > 1:
        return fact[0], fact[-1]
    else:
        return fact[0], True

In [20]:
def add_fact(line):
    var, val = convert_fact(line)
    facts[var] = val

In [21]:
with open('/content/facts.txt', 'r') as fact_file:
    for line in fact_file:
        add_fact(line)

In [22]:
facts

{'seeds': '0', 'diameter': '7', 'skin_smell': True, 'color': 'orange'}

In [16]:
def convert_rule(line):
    rule = line.strip().split(' THEN ')
    if_part = rule[0].strip()[3:]
    then_part = rule[1].strip()
    return if_part, then_part

In [23]:
def add_rule(line):
    if_part, then_part = convert_rule(line)
    rules[if_part] = then_part

In [24]:
with open('/content/rules.txt', 'r') as rule_file:
    for line in rule_file:
        add_rule(line)

In [25]:
rules

{'shape is long AND color is yellow': 'fruit is banana',
 'shape is round AND color is red AND size is medium': 'fruit is apple',
 'shape is round AND color is red AND size is small': 'fruit is cherry',
 'skin_smell': 'perfumed',
 'fruit is lemon OR fruit is orange OR fruit is pomelo OR fruit is grapefruit': 'citrus_fruit',
 'size is medium AND color is yellow AND perfumed': 'fruit is lemon',
 'size is medium AND color is green': 'fruit is kiwi',
 'size is big AND perfumed AND color is orange AND citrus_fruit': 'fruit is grapefruit',
 'perfumed AND color is orange AND size is medium': 'fruit is orange',
 'perfumed AND color is red AND size is small AND seeds = 0': 'fruit is strawberry',
 'diameter < 2': 'size is small',
 'diameter > 10': 'size is big',
 'diameter > 2 AND diameter < 10': 'size is medium'}

In [26]:
def check_rule(if_part,then_part):
    check_list = []
    And = False
    Or = False
    if 'AND' in if_part:
        And = True
        if_part = if_part.split(' AND ')
    elif 'OR' in if_part:
        Or = True
        if_part = if_part.split(' OR ')

    for part in if_part:
        var, val = convert_fact(part)
        if facts.get(var) == val:
            check_list.append(True)
        else:
            check_list.append(False)

    if And:
        if all(check_list):
            return True
        else:
            return False
    elif Or:
        if any(check_list):
            return True
        else:
            return False
    else:
        return check_list[0]